### Graham Kerr
#### graham.s.kerr@NASA.gov; kerrg@cua.edu

<H1 font size="+3" style="color:red">
O I in Flares<br>
-- IRIS SJI - Align<br>
</H1>

<b>This notebook will</b>
> - Aligns the SJI data to account for drift





---
### <b style="color:blue"> Some set up </b>

***Import Modules***

In [ ]:
##
## Import various modules -- can tidy up later
##
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import os
import re
from scipy import io
import pickle
import cmocean
import copy
import iris_lmsalpy
import astropy
import pandas as pd
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs')
import utils
from scipy.ndimage import fourier_shift
import scipy.ndimage 



---
### <b style="color:blue"> Read in the data </b>


In [ ]:
##
## Load the SJI data  
##
dir1 = '/Users/gskerr1/Documents/Research/Melissa_OI_IRIS/2014_10_25_1500/'
file_sji = dir1+'IRIS_SJI_2014_Oct_25_exptimecorr.pkl'
with open(file_sji, 'rb') as output:  
    sji_dict = pickle.load(output)
    
sji_dict_aligned = copy.deepcopy(sji_dict)



In [ ]:
sji_dict.keys()

---
### <b style="color:blue"> Some set up </b>


***Some dimensions***

In [ ]:
##
## Some dimensions
##
ny_1330 = (sji_dict['sji_1330'].shape)[0]
nx_1330 = (sji_dict['sji_1330'].shape)[1]
nt_1330 = (sji_dict['sji_1330'].shape)[2]

ny_2832 = (sji_dict['sji_2832'].shape)[0]
nx_2832 = (sji_dict['sji_2832'].shape)[1]
nt_2832 = (sji_dict['sji_2832'].shape)[2]

ny_2796 = (sji_dict['sji_2796'].shape)[0]
nx_2796 = (sji_dict['sji_2796'].shape)[1]
nt_2796 = (sji_dict['sji_2796'].shape)[2]

In [ ]:
shifted_data_1330 = np.zeros([ny_1330,nx_1330,nt_1330],dtype=np.float)
shifted_data_2832 = np.zeros([ny_2832,nx_1330,nt_2832],dtype=np.float)
shifted_data_2796 = np.zeros([ny_2796,nx_1330,nt_2796],dtype=np.float)

In [ ]:
##
## Reference times for SJI alignment
##

## only grab indces with times that aren't messed up in the header
inds_2832 = np.where(np.array(sji_dict['time_2832']) != '-- ::.')[0]
inds_2796 = np.where(np.array(sji_dict['time_2796']) != '-- ::.')[0]
inds_1330 = np.where(np.array(sji_dict['time_1330']) != '-- ::.')[0]

## reference time to shift data to
ref_time = '2014-10-25 16:30:00.00'

ref_ind_sji2832_tmp = (np.where(pd.to_datetime(np.array(sji_dict['time_2832'])[inds_2832]) == utils.nearest(pd.to_datetime(np.array(sji_dict['time_2832'])[inds_2832]), pd.to_datetime(ref_time))))[0][0]
ref_ind_sji2832 = inds_2832[ref_ind_sji2832_tmp]
ref_time_sji2832 = sji_dict['time_2832'][ref_ind_sji2832]

ref_ind_sji2796_tmp = (np.where(pd.to_datetime(np.array(sji_dict['time_2796'])[inds_2796]) == utils.nearest(pd.to_datetime(np.array(sji_dict['time_2796'])[inds_2796]), pd.to_datetime(ref_time))))[0][0]
ref_ind_sji2796 = inds_2796[ref_ind_sji2796_tmp]
ref_time_sji2796 = sji_dict['time_2796'][ref_ind_sji2796]

ref_ind_sji1330_tmp = (np.where(pd.to_datetime(np.array(sji_dict['time_1330'])[inds_1330]) == utils.nearest(pd.to_datetime(np.array(sji_dict['time_1330'])[inds_1330]), pd.to_datetime(ref_time))))[0][0]
ref_ind_sji2796 = inds_2796[ref_ind_sji1330_tmp]
ref_time_sji1330 = sji_dict['time_1330'][ref_ind_sji1330]

In [ ]:
print(ref_time_sji2796)
print(ref_time_sji2832)
print(ref_time_sji1330)

In [ ]:
##
## Extract the x and y coordinates
## 

ycoord_sji_1330 = np.zeros([ny_1330,nt_1330],dtype=np.float64)
xcoord_sji_1330 = np.zeros([nx_1330,nt_1330],dtype=np.float64)

for ind_sji in range(nt_1330):
   
    xmin_sji_1330= np.min(sji_dict['xcen_1330'][ind_sji]-sji_dict['hdr_1330_ext0']['CDELT1']*(sji_dict['hdr_1330_ext0']['NAXIS1']-1.e0)/2.e0)#-dx_131/2.e0
    xmax_sji_1330= np.max(sji_dict['xcen_1330'][ind_sji]+sji_dict['hdr_1330_ext0']['CDELT1']*(sji_dict['hdr_1330_ext0']['NAXIS1']-1.e0)/2.e0)#+dx_131/2.e0
    ymin_sji_1330= np.min(sji_dict['ycen_1330'][ind_sji]-sji_dict['hdr_1330_ext0']['CDELT2']*(sji_dict['hdr_1330_ext0']['NAXIS2']-1.e0)/2.e0)#-dx_131/2.e0
    ymax_sji_1330= np.max(sji_dict['ycen_1330'][ind_sji]+sji_dict['hdr_1330_ext0']['CDELT2']*(sji_dict['hdr_1330_ext0']['NAXIS2']-1.e0)/2.e0)#+dx_131/2.e0
 
   
    xcoord_sji_1330[:,ind_sji] = np.linspace(xmin_sji_1330,xmax_sji_1330,num=sji_dict['hdr_1330_ext0']['NAXIS1']) 
    ycoord_sji_1330[:,ind_sji] = np.linspace(ymin_sji_1330,ymax_sji_1330,num=sji_dict['hdr_1330_ext0']['NAXIS2']) 



In [ ]:
ycoord_sji_2796 = np.zeros([ny_2796,nt_2796],dtype=np.float64)
xcoord_sji_2796 = np.zeros([nx_2796,nt_2796],dtype=np.float64)

for ind_sji in range(nt_2796):
   
    xmin_sji_2796= np.min(sji_dict['xcen_2796'][ind_sji]-sji_dict['hdr_2796_ext0']['CDELT1']*(sji_dict['hdr_2796_ext0']['NAXIS1']-1.e0)/2.e0)#-dx_131/2.e0
    xmax_sji_2796= np.max(sji_dict['xcen_2796'][ind_sji]+sji_dict['hdr_2796_ext0']['CDELT1']*(sji_dict['hdr_2796_ext0']['NAXIS1']-1.e0)/2.e0)#+dx_131/2.e0
    ymin_sji_2796= np.min(sji_dict['ycen_2796'][ind_sji]-sji_dict['hdr_2796_ext0']['CDELT2']*(sji_dict['hdr_2796_ext0']['NAXIS2']-1.e0)/2.e0)#-dx_131/2.e0
    ymax_sji_2796= np.max(sji_dict['ycen_2796'][ind_sji]+sji_dict['hdr_2796_ext0']['CDELT2']*(sji_dict['hdr_2796_ext0']['NAXIS2']-1.e0)/2.e0)#+dx_131/2.e0
 
   
    xcoord_sji_2796[:,ind_sji] = np.linspace(xmin_sji_2796,xmax_sji_2796,num=sji_dict['hdr_2796_ext0']['NAXIS1']) 
    ycoord_sji_2796[:,ind_sji] = np.linspace(ymin_sji_2796,ymax_sji_2796,num=sji_dict['hdr_2796_ext0']['NAXIS2']) 



In [ ]:
ycoord_sji_2832 = np.zeros([ny_2832,nt_2832],dtype=np.float64)
xcoord_sji_2832 = np.zeros([nx_2832,nt_2832],dtype=np.float64)

for ind_sji in range(nt_2832):
   
    xmin_sji_2832= np.min(sji_dict['xcen_2832'][ind_sji]-sji_dict['hdr_2832_ext0']['CDELT1']*(sji_dict['hdr_2832_ext0']['NAXIS1']-1.e0)/2.e0)#-dx_131/2.e0
    xmax_sji_2832= np.max(sji_dict['xcen_2832'][ind_sji]+sji_dict['hdr_2832_ext0']['CDELT1']*(sji_dict['hdr_2832_ext0']['NAXIS1']-1.e0)/2.e0)#+dx_131/2.e0
    ymin_sji_2832= np.min(sji_dict['ycen_2832'][ind_sji]-sji_dict['hdr_2832_ext0']['CDELT2']*(sji_dict['hdr_2832_ext0']['NAXIS2']-1.e0)/2.e0)#-dx_131/2.e0
    ymax_sji_2832= np.max(sji_dict['ycen_2832'][ind_sji]+sji_dict['hdr_2832_ext0']['CDELT2']*(sji_dict['hdr_2832_ext0']['NAXIS2']-1.e0)/2.e0)#+dx_131/2.e0
 
   
    xcoord_sji_2832[:,ind_sji] = np.linspace(xmin_sji_2832,xmax_sji_2832,num=sji_dict['hdr_2832_ext0']['NAXIS1']) 
    ycoord_sji_2832[:,ind_sji] = np.linspace(ymin_sji_2832,ymax_sji_2832,num=sji_dict['hdr_2832_ext0']['NAXIS2']) 



In [ ]:
##
## Compute the sub-pixel shifts and perform that shift
##

shift_arc_1330 = np.zeros([nt_1330],dtype=np.float64)

for tind in range(0,nt_1330):
        
        shift_px_1330 = ((ycoord_sji_1330[:,ref_ind_sji1330] - ycoord_sji_1330[:,tind])/sji_dict['hdr_1330_ext0']['CDELT1'])[0]
        shift_arc_1330[tind] = shift_px_1330*sji_dict['hdr_1330_ext0']['CDELT1']
        shifted_image_1330 = scipy.ndimage.shift(sji_dict['sji_1330'][:,:,tind],[shift_px_1330,0])
#         shifted_image_1330 = fourier_shift(np.fft.fftn(sji_dict['sji_1330'][:,:,tind]), [shift_px_1330,0])
#         shifted_image_1330 = np.fft.ifftn(shifted_image_1330)
#         shifted_data_1330[:,:,tind] = shifted_image_1330.real 
        shifted_data_1330[:,:,tind] = shifted_image_1330
        sji_dict_aligned['ycen_1330'][tind] = sji_dict['ycen_1330'][tind] + shift_px_1330*sji_dict['hdr_1330_ext0']['CDELT1']


sji_dict_aligned['sji_1330'] = shifted_data_1330




In [ ]:
sji_dict_aligned['ycen_1330']

In [ ]:
shift_arc_2796 = np.zeros([nt_2796],dtype=np.float64)

for tind in range(0,nt_2796):
        
        shift_px_2796 = ((ycoord_sji_1330[:,ref_ind_sji1330] - ycoord_sji_2796[:,tind])/sji_dict['hdr_2796_ext0']['CDELT1'])[0]
        shift_arc_2796[tind] = shift_px_2796*sji_dict['hdr_2796_ext0']['CDELT1']
        shifted_image_2796 = scipy.ndimage.shift(sji_dict['sji_2796'][:,:,tind],[shift_px_2796,0])
#         shifted_image_2796 = fourier_shift(np.fft.fftn(sji_dict['sji_2796'][:,:,tind]), [shift_px_2796,0])
#         shifted_image_2796 = np.fft.ifftn(shifted_image_2796)
#         shifted_data_2796[:,:,tind] = shifted_image_2796.real 
        shifted_data_2796[:,:,tind] = shifted_image_2796
        sji_dict_aligned['ycen_2796'][tind] = sji_dict['ycen_2796'][tind] + shift_px_2796*sji_dict['hdr_2796_ext0']['CDELT1']


sji_dict_aligned['sji_2796'] = shifted_data_2796



In [ ]:
shift_arc_2832 = np.zeros([nt_2832],dtype=np.float64)
shift_px_2832 = np.zeros([nt_2832],dtype=np.float64)


for tind in range(0,nt_2832):
        
        shift_px_2832[tind] = ((ycoord_sji_1330[:,ref_ind_sji1330] - ycoord_sji_2832[:,tind])/sji_dict['hdr_2832_ext0']['CDELT1'])[0]
        shift_arc_2832[tind] = shift_px_2832[tind]*sji_dict['hdr_2832_ext0']['CDELT1']
        shifted_image_2832 = scipy.ndimage.shift(sji_dict['sji_2832'][:,:,tind],[shift_px_2832[tind],0])
#         shifted_image_2832 = fourier_shift(np.fft.fftn(sji_dict['sji_2832'][:,:,tind]), [shift_px_2832,0])
#         shifted_image_2832 = np.fft.ifftn(shifted_image_2832)
#         shifted_data_2832[:,:,tind] = shifted_image_2832.real 
        shifted_data_2832[:,:,tind] = shifted_image_2832
        sji_dict_aligned['ycen_2832'][tind] = sji_dict['ycen_2832'][tind] + shift_px_2832[tind]*sji_dict['hdr_2832_ext0']['CDELT1']


sji_dict_aligned['sji_2832'] = shifted_data_2832



In [ ]:
plt.plot(shift_px_2832)
plt.ylim(-5,5)

In [ ]:
##
## What is the range of shifts?
##
plt.plot(shift_arc_1330)
plt.plot(shift_arc_2796)
plt.plot(shift_arc_2832)

plt.ylim(-2.5, 2.5)

In [ ]:
plt.plot(sji_dict['ycen_1330'])
plt.plot(sji_dict['ycen_2796'])
plt.plot(sji_dict['ycen_2832'])
plt.ylim(-320, -315)

In [ ]:
## 
## Set ~1" top and bottom to zero 
##

sji_dict_aligned['sji_1330'][0:4,:,:] = np.nan
sji_dict_aligned['sji_1330'][-4:,:,:] = np.nan

sji_dict_aligned['sji_2796'][0:4,:,:] = np.nan
sji_dict_aligned['sji_2796'][-4:,:,:] = np.nan

sji_dict_aligned['sji_2832'][0:4,:,:] = np.nan
sji_dict_aligned['sji_2832'][-4:,:,:] = np.nan

***Plot the aligned dataset***

In [ ]:
savepng = True

dirout = '/Users/gskerr1/Documents/Research/Melissa_OI_IRIS/2014_10_25_1500/'
fname_out = dirout+'IRISSJI_1330A_2015_Oct_25_aligned_V4'

data = copy.deepcopy(sji_dict_aligned['sji_1330'])**0.25

index = sji_dict['hdr_1330_ext0']

t1 = 0
t2 = sji_dict_aligned['sji_1330'].shape[2]-1

cmap1330 = cmocean.cm.solar

xrange = [360,460]
yrange = [-370, -260]

xsel1_wl = 0
xsel2_wl = sji_dict_aligned['sji_1330'].shape[1]-1
ysel1_wl = 0
ysel2_wl = sji_dict_aligned['sji_1330'].shape[0]-1

deltaxr = np.abs(xrange[0]-xrange[-1])
deltayr = np.abs(yrange[0]-yrange[-1])

aspect = deltaxr/deltayr

xsize = 4
ysize = 4

dx = 0.75
dy = dx/aspect
y1 = 0.05
x1 = 0.05
sx = 0.15


counter = 0
for tind in range(t1,t2):
    
    xmin_sji= np.min(sji_dict['xcen_1330'][tind]-sji_dict['hdr_1330_ext0']['CDELT1']*(sji_dict['hdr_1330_ext0']['NAXIS1']-1.e0)/2.e0)
    xmax_sji= np.max(sji_dict['xcen_1330'][tind]+sji_dict['hdr_1330_ext0']['CDELT1']*(sji_dict['hdr_1330_ext0']['NAXIS1']-1.e0)/2.e0)
    xcoord_sji = np.linspace(xmin_sji,xmax_sji,num=sji_dict['hdr_1330_ext0']['NAXIS1']) 
    ymin_sji= np.min(sji_dict['ycen_1330'][tind]-sji_dict['hdr_1330_ext0']['CDELT2']*(sji_dict['hdr_1330_ext0']['NAXIS2']-1.e0)/2.e0)
    ymax_sji= np.max(sji_dict['ycen_1330'][tind]+sji_dict['hdr_1330_ext0']['CDELT2']*(sji_dict['hdr_1330_ext0']['NAXIS2']-1.e0)/2.e0)
    ycoord_sji = np.linspace(ymin_sji,ymax_sji,num=sji_dict['hdr_1330_ext0']['NAXIS2']) 

    fig = plt.figure(figsize=(xsize, ysize))
    ax1 = fig.add_axes([x1, y1, dx, dy])


    ax1.set_xlim(xrange[0], xrange[-1])
    ax1.set_ylim(yrange[0], yrange[-1])

    ax1.set_ylabel('Solar - Y [arcsec]')
    ax1.set_xlabel('Solar - X [arcsec]')
    ax1.xaxis.set_tick_params(rotation=20)
    ax1.tick_params(which='both', direction='inout')

    im = ax1.imshow(data[:,:,tind],
                    cmap=cmap1330, 
                    origin='lower',interpolation='nearest', aspect='equal',
                    extent = [xcoord_sji[xsel1_wl],xcoord_sji[xsel2_wl],
                              ycoord_sji[ysel1_wl],ycoord_sji[ysel2_wl]])


#     ax1.text(1.05, 0.95, 'IRIS/SJI 1400A', color = 'black', transform = ax1.transAxes, fontsize = 15)
#     ax1.text(1.05, 0.90, sji_dict['time_1400'][tind]+'UT'  , color = 'black',transform=ax1.transAxes,fontsize=15)     
    ax1.set_title('IRIS/SJI 1330A\n'+sji_dict['time_1330'][tind]+'UT')
    ax1.axvline(x=sji_dict['lcslit_1330'][tind], color = 'white', linewidth=1, linestyle='--', zorder = 20)

    if savepng == True:
        plt.savefig(fname_out+'_%04d.png' %(counter), format='png', bbox_inches = 'tight', dpi=300)
    
    counter+= 1
    
    plt.show()

In [ ]:
savepng = True

dirout = '/Users/gskerr1/Documents/Research/Melissa_OI_IRIS/2014_10_25_1500/'
fname_out = dirout+'IRISSJI_1330A_2015_Oct_25_aligned_V3'

data = copy.deepcopy(sji_dict_aligned['sji_1330'])**0.25

index = sji_dict['hdr_1330_ext0']

t1 = 0
t2 = sji_dict_aligned['sji_1330'].shape[2]-1

cmap1330 = cmocean.cm.solar

xrange = [360,460]
yrange = [-370, -260]

xsel1_wl = 0
xsel2_wl = sji_dict_aligned['sji_1330'].shape[1]-1
ysel1_wl = 0
ysel2_wl = sji_dict_aligned['sji_1330'].shape[0]-1

deltaxr = np.abs(xrange[0]-xrange[-1])
deltayr = np.abs(yrange[0]-yrange[-1])

aspect = deltaxr/deltayr

xsize = 4
ysize = 4

dx = 0.75
dy = dx/aspect
y1 = 0.05
x1 = 0.05
sx = 0.15


counter = 0
for tind in range(t1,t2):
    
    xmin_sji= np.min(sji_dict_aligned['xcen_1330'][tind]-sji_dict_aligned['hdr_1330_ext0']['CDELT1']*(sji_dict_aligned['hdr_1330_ext0']['NAXIS1']-1.e0)/2.e0)
    xmax_sji= np.max(sji_dict_aligned['xcen_1330'][tind]+sji_dict_aligned['hdr_1330_ext0']['CDELT1']*(sji_dict_aligned['hdr_1330_ext0']['NAXIS1']-1.e0)/2.e0)
    xcoord_sji = np.linspace(xmin_sji,xmax_sji,num=sji_dict_aligned['hdr_1330_ext0']['NAXIS1']) 
    ymin_sji= np.min(sji_dict_aligned['ycen_1330'][tind]-sji_dict_aligned['hdr_1330_ext0']['CDELT2']*(sji_dict_aligned['hdr_1330_ext0']['NAXIS2']-1.e0)/2.e0)
    ymax_sji= np.max(sji_dict_aligned['ycen_1330'][tind]+sji_dict_aligned['hdr_1330_ext0']['CDELT2']*(sji_dict_aligned['hdr_1330_ext0']['NAXIS2']-1.e0)/2.e0)
    ycoord_sji = np.linspace(ymin_sji,ymax_sji,num=sji_dict_aligned['hdr_1330_ext0']['NAXIS2']) 

    fig = plt.figure(figsize=(xsize, ysize))
    ax1 = fig.add_axes([x1, y1, dx, dy])


#     ax1.set_xlim(xrange[0], xrange[-1])
#     ax1.set_ylim(yrange[0], yrange[-1])

    ax1.set_ylabel('Solar - Y [pixel]')
    ax1.set_xlabel('Solar - X [pixel]')
    ax1.xaxis.set_tick_params(rotation=20)
    ax1.tick_params(which='both', direction='inout')

    im = ax1.imshow(data[:,:,tind],
                    cmap=cmap1330, 
                    origin='lower',interpolation='nearest', aspect='equal')
#                     extent = [xcoord_sji[xsel1_wl],xcoord_sji[xsel2_wl],
#                               ycoord_sji[ysel1_wl],ycoord_sji[ysel2_wl]])


#     ax1.text(1.05, 0.95, 'IRIS/SJI 1400A', color = 'black', transform = ax1.transAxes, fontsize = 15)
#     ax1.text(1.05, 0.90, sji_dict['time_1400'][tind]+'UT'  , color = 'black',transform=ax1.transAxes,fontsize=15)     
    ax1.set_title('IRIS/SJI 1330A\n'+sji_dict['time_1330'][tind]+'UT')
    ax1.axvline(x=sji_dict['lcslit_1330'][tind], color = 'white', linewidth=1, linestyle='--', zorder = 20)

    if savepng == True:
        plt.savefig(fname_out+'_%04d.png' %(counter), format='png', bbox_inches = 'tight', dpi=300)
    
    counter+= 1
    
    plt.show()

In [ ]:
savepng = True

dirout = '/Users/gskerr1/Documents/Research/Melissa_OI_IRIS/2014_10_25_1500/'
fname_out = dirout+'IRISSJI_2796A_2015_Oct_25_aligned'

data = copy.deepcopy(sji_dict_aligned['sji_2796'])**0.25

index = sji_dict_aligned['hdr_2796_ext0']

t1 = 0
t2 = sji_dict_aligned['sji_2796'].shape[2]-1

cmap2796 = cmocean.cm.thermal

xrange = [360,460]
yrange = [-370, -260]

xsel1_wl = 0
xsel2_wl = sji_dict_aligned['sji_2796'].shape[1]-1
ysel1_wl = 0
ysel2_wl = sji_dict_aligned['sji_2796'].shape[0]-1

deltaxr = np.abs(xrange[0]-xrange[-1])
deltayr = np.abs(yrange[0]-yrange[-1])

aspect = deltaxr/deltayr

xsize = 4
ysize = 4

dx = 0.75
dy = dx/aspect
y1 = 0.05
x1 = 0.05
sx = 0.15


counter = 0
for tind in range(t1,t2):
    
    xmin_sji= np.min(sji_dict_aligned['xcen_2796'][tind]-sji_dict_aligned['hdr_2796_ext0']['CDELT1']*(sji_dict_aligned['hdr_2796_ext0']['NAXIS1']-1.e0)/2.e0)
    xmax_sji= np.max(sji_dict_aligned['xcen_2796'][tind]+sji_dict_aligned['hdr_2796_ext0']['CDELT1']*(sji_dict_aligned['hdr_2796_ext0']['NAXIS1']-1.e0)/2.e0)
    xcoord_sji = np.linspace(xmin_sji,xmax_sji,num=sji_dict_aligned['hdr_2796_ext0']['NAXIS1']) 
    ymin_sji= np.min(sji_dict_aligned['ycen_2796'][tind]-sji_dict_aligned['hdr_2796_ext0']['CDELT2']*(sji_dict_aligned['hdr_2796_ext0']['NAXIS2']-1.e0)/2.e0)
    ymax_sji= np.max(sji_dict_aligned['ycen_2796'][tind]+sji_dict_aligned['hdr_2796_ext0']['CDELT2']*(sji_dict_aligned['hdr_2796_ext0']['NAXIS2']-1.e0)/2.e0)
    ycoord_sji = np.linspace(ymin_sji,ymax_sji,num=sji_dict_aligned['hdr_2796_ext0']['NAXIS2']) 

    fig = plt.figure(figsize=(xsize, ysize))
    ax1 = fig.add_axes([x1, y1, dx, dy])


    ax1.set_xlim(xrange[0], xrange[-1])
    ax1.set_ylim(yrange[0], yrange[-1])

    ax1.set_ylabel('Solar - Y [arcsec]')
    ax1.set_xlabel('Solar - X [arcsec]')
    ax1.xaxis.set_tick_params(rotation=20)
    ax1.tick_params(which='both', direction='inout')

    im = ax1.imshow(data[:,:,tind],
                    cmap=cmap2796, 
                    origin='lower',interpolation='nearest', aspect='equal',
                    extent = [xcoord_sji[xsel1_wl],xcoord_sji[xsel2_wl],
                              ycoord_sji[ysel1_wl],ycoord_sji[ysel2_wl]])


#     ax1.text(1.05, 0.95, 'IRIS/SJI 1400A', color = 'black', transform = ax1.transAxes, fontsize = 15)
#     ax1.text(1.05, 0.90, sji_dict['time_1400'][tind]+'UT'  , color = 'black',transform=ax1.transAxes,fontsize=15)     
    ax1.set_title('IRIS/SJI 2796A\n'+sji_dict['time_2796'][tind]+'UT')
    ax1.axvline(x=sji_dict['lcslit_2796'][tind], color = 'white', linewidth=1, linestyle='--', zorder = 20)

    if savepng == True:
        plt.savefig(fname_out+'_%04d.png' %(counter), format='png', bbox_inches = 'tight', dpi=300)
    
    counter+= 1
    
    plt.show()

In [ ]:
savepng = True

dirout = '/Users/gskerr1/Documents/Research/Melissa_OI_IRIS/2014_10_25_1500/'
fname_out = dirout+'IRISSJI_2832A_2015_Oct_25_aligned'

data = copy.deepcopy(sji_dict_aligned['sji_2832'])**0.25

index = sji_dict_aligned['hdr_2832_ext0']

t1 = 0
t2 = sji_dict_aligned['sji_2832'].shape[2]-1

cmap2832 = cmocean.cm.haline

xrange = [360,460]
yrange = [-370, -260]

xsel1_wl = 0
xsel2_wl = sji_dict_aligned['sji_2832'].shape[1]-1
ysel1_wl = 0
ysel2_wl = sji_dict_aligned['sji_2832'].shape[0]-1

deltaxr = np.abs(xrange[0]-xrange[-1])
deltayr = np.abs(yrange[0]-yrange[-1])

aspect = deltaxr/deltayr

xsize = 4
ysize = 4

dx = 0.75
dy = dx/aspect
y1 = 0.05
x1 = 0.05
sx = 0.15


counter = 0
for tind in range(t1,t2):
    
    xmin_sji= np.min(sji_dict_aligned['xcen_2832'][tind]-sji_dict_aligned['hdr_2832_ext0']['CDELT1']*(sji_dict_aligned['hdr_2832_ext0']['NAXIS1']-1.e0)/2.e0)
    xmax_sji= np.max(sji_dict_aligned['xcen_2832'][tind]+sji_dict_aligned['hdr_2832_ext0']['CDELT1']*(sji_dict_aligned['hdr_2832_ext0']['NAXIS1']-1.e0)/2.e0)
    xcoord_sji = np.linspace(xmin_sji,xmax_sji,num=sji_dict_aligned['hdr_2832_ext0']['NAXIS1']) 
    ymin_sji= np.min(sji_dict_aligned['ycen_2832'][tind]-sji_dict_aligned['hdr_2832_ext0']['CDELT2']*(sji_dict_aligned['hdr_2832_ext0']['NAXIS2']-1.e0)/2.e0)
    ymax_sji= np.max(sji_dict_aligned['ycen_2832'][tind]+sji_dict_aligned['hdr_2832_ext0']['CDELT2']*(sji_dict_aligned['hdr_2832_ext0']['NAXIS2']-1.e0)/2.e0)
    ycoord_sji = np.linspace(ymin_sji,ymax_sji,num=sji_dict_aligned['hdr_2832_ext0']['NAXIS2']) 

    fig = plt.figure(figsize=(xsize, ysize))
    ax1 = fig.add_axes([x1, y1, dx, dy])


    ax1.set_xlim(xrange[0], xrange[-1])
    ax1.set_ylim(yrange[0], yrange[-1])

    ax1.set_ylabel('Solar - Y [arcsec]')
    ax1.set_xlabel('Solar - X [arcsec]')
    ax1.xaxis.set_tick_params(rotation=20)
    ax1.tick_params(which='both', direction='inout')

    im = ax1.imshow(data[:,:,tind],
                    cmap=cmap2832, 
                    origin='lower',interpolation='nearest', aspect='equal',
                    extent = [xcoord_sji[xsel1_wl],xcoord_sji[xsel2_wl],
                              ycoord_sji[ysel1_wl],ycoord_sji[ysel2_wl]])


#     ax1.text(1.05, 0.95, 'IRIS/SJI 1400A', color = 'black', transform = ax1.transAxes, fontsize = 15)
#     ax1.text(1.05, 0.90, sji_dict['time_1400'][tind]+'UT'  , color = 'black',transform=ax1.transAxes,fontsize=15)     
    ax1.set_title('IRIS/SJI 2832A\n'+sji_dict['time_2832'][tind]+'UT')
    ax1.axvline(x=sji_dict['lcslit_2832'][tind], color = 'white', linewidth=1, linestyle='--', zorder = 20)

    if savepng == True:
        plt.savefig(fname_out+'_%04d.png' %(counter), format='png', bbox_inches = 'tight', dpi=300)
    
    counter+= 1
    
    plt.show()